# DEM Retriever

This notebook is a secondary notebook, supposed to get the digital elevation model from opentopography.org

## Getting DEM-Data from [opentopography](https://portal.opentopography.org/apidocs/#/Public/getGlobalDem)
To run this notebook you will need a (free) API key from *opentopography.org*. 
I’ve got myself one and put it into the environment variables.

In [60]:
# All the imports

# For getting the DEM-Data (from opentopography.org) 
import requests

# Handling the API-Key (hidden from this repo)
from dotenv import load_dotenv

# Filesystem, JSON
import os
import json
from io import BytesIO # To be able to route the API response to a file-like object that rasterio can use

# Rasterio for handling GeoTIFFs
import rasterio

# Math may not be missing
import math
import numpy as np

# Regex for updating filenames
import re

# For earth related numbers
from pyproj import Geod
geod = Geod(ellps="WGS84")

# print(f"Equatorial radius (a): {geod.a / 1000:.1f} km")  # 6378.1 km
# print(f"Polar radius (b): {geod.b / 1000:.1f} km")      # 6356.8 km



In [2]:
# Let this notebook load the API key from the .env file
load_dotenv()

# Create a dictionary for settings that are not saved
external_settings = {}
external_settings["opentopo_api_key"] = os.getenv("OPENTOPO_API_KEY")

### Hint: API Key
If you want to use your own API-Key, you’ll have to assign it in the following code block. Remember: You will have to get your own API key from opentopography.org to make this work.

In [12]:

# Hint! If you want to use your own API-Key, you’ll have to assign it here:
# external_settings["opentopo_api_key"]

## Getting the DEM-Data
For this example we’ll use the EU_DTM dataset which is very accurate for europe and resembles bare earth heights.

In [ ]:
# Create a variable for settings that can later be saved as a JSON-file
internal_settings = {}

internal_settings["geo"] = {}
internal_settings["geo"]["demtype"] = "COP90"
internal_settings["geo"]["west"] = 6.5
internal_settings["geo"]["south"] = 46.93
internal_settings["geo"]["east"] = 8.6
internal_settings["geo"]["north"] = 48.02

internal_settings["geo"]["outputFormat"] = "GTiff"

# When the average difference of the given bounds to those of an existing geotiff image 
# is less than this threshold, the GeoTIFF is considered already existent. 
# This is to prevent another api call 
internal_settings["geo"]["tolerance"] = 0.5 # In degrees. Could be set lower if the regions become very small

internal_settings["fft"] = {}
internal_settings["fft"]["tile_size"] = 1.75 # in km
internal_settings["fft"]["tile_size"] = 26.9 # in km JUST FOR TESTING REMOVE LATER

internal_settings["files"] = {}
internal_settings["files"]["intermediary_folder"] = "intermediary"

# Important: Geotiffs have to have the extension "TIF" to be readable by google earth pro...x
internal_settings["files"]["geotiff"] = "geotiff COORDINATES.tif" 


# Generate temporary settings
temporary_settings = {}

temporary_settings["dem_coordinate_string"] = ", ".join([f"{internal_settings["geo"]["west"]:05.1f}",
                                                         f"{internal_settings["geo"]["south"]:05.1f}",
                                                         f"{internal_settings["geo"]["east"]:05.1f}",
                                                         f"{internal_settings["geo"]["north"]:05.1f}"])

temporary_settings["dem_file_location"] = os.path.join(internal_settings["files"]
                                                       ["intermediary_folder"],
                                                       re.sub(r"COORDINATES", temporary_settings["dem_coordinate_string"], internal_settings["files"]["geotiff"]))

temporary_settings["dem_already_existent"] = False

temporary_settings["dem_latitude_max"] = max(abs(internal_settings["geo"]["south"]), abs(internal_settings["geo"]["north"]))
temporary_settings["dem_latitude_min"] = min(abs(internal_settings["geo"]["south"]), abs(internal_settings["geo"]["north"]))

# Calculate the height of the DEM image in km
temporary_settings["dem_size_height"] = (abs(internal_settings["geo"]["south"] - internal_settings["geo"]["north"]) / 360.0) * math.pi * 2.0 * geod.b / 1000.0; 

# Calculate the DEM images width at the border that is closer to the equator
temporary_settings["dem_size_width_max"] = (abs(internal_settings["geo"]["west"] - 
                                           internal_settings["geo"]["east"]) / 360.0) * math.pi * 2.0 * geod.a * math.cos( math.radians(temporary_settings["dem_latitude_min"])) / 1000.0 ; 

# Calculate the DEM images width at the border that is further from the equator
temporary_settings["dem_size_width_min"] = (abs(internal_settings["geo"]["west"] - 
                                           internal_settings["geo"]["east"]) / 360.0) *  math.pi * 2.0 * geod.a * math.cos(math.radians(temporary_settings["dem_latitude_max"])) / 1000.0 ; 


In [46]:
print(json.dumps(temporary_settings, indent = 4))

{
    "dem_coordinate_string": "006.5, 046.9, 008.6, 048.0",
    "dem_file_location": "intermediary/geotiff 006.5, 046.9, 008.6, 048.0.tif",
    "dem_already_existent": false,
    "dem_latitude_max": 48.02,
    "dem_latitude_min": 46.93,
    "dem_size_height": 120.93142080290929,
    "dem_size_width_max": 159.64015063229772,
    "dem_size_width_min": 156.36263336254996
}


In [81]:

num_tiles_y = temporary_settings["dem_size_height"] / internal_settings["fft"]["tile_size"]
print(num_tiles_y)

relative_size_y = 1 / num_tiles_y
print(relative_size_y)

rest_tiles_y = num_tiles_y % 1;
print(rest_tiles_y)

relative_start_y = (rest_tiles_y / 2) / num_tiles_y
print(relative_start_y)

relative_end_y = 1 - relative_start_y

firstrow_center = np.array([relative_start_y / 2])

lastrow_center = np.array([1 - (relative_start_y / 2)])

all_other_rows_center = np.linspace(relative_start_y + (relative_size_y / 2), relative_end_y - (relative_size_y / 2), math.floor(num_tiles_y), endpoint=True)


allrows_centers = np.concatenate((firstrow_center, all_other_rows_center, lastrow_center))

print(allrows_centers)


69.10366903023387
0.014471011655871488
0.10366903023387408
0.0007500978724336428
[3.75048936e-04 7.98560370e-03 2.24566154e-02 3.69276270e-02
 5.13986387e-02 6.58696503e-02 8.03406620e-02 9.48116736e-02
 1.09282685e-01 1.23753697e-01 1.38224709e-01 1.52695720e-01
 1.67166732e-01 1.81637744e-01 1.96108755e-01 2.10579767e-01
 2.25050779e-01 2.39521790e-01 2.53992802e-01 2.68463814e-01
 2.82934825e-01 2.97405837e-01 3.11876848e-01 3.26347860e-01
 3.40818872e-01 3.55289883e-01 3.69760895e-01 3.84231907e-01
 3.98702918e-01 4.13173930e-01 4.27644942e-01 4.42115953e-01
 4.56586965e-01 4.71057977e-01 4.85528988e-01 5.00000000e-01
 5.14471012e-01 5.28942023e-01 5.43413035e-01 5.57884047e-01
 5.72355058e-01 5.86826070e-01 6.01297082e-01 6.15768093e-01
 6.30239105e-01 6.44710117e-01 6.59181128e-01 6.73652140e-01
 6.88123152e-01 7.02594163e-01 7.17065175e-01 7.31536186e-01
 7.46007198e-01 7.60478210e-01 7.74949221e-01 7.89420233e-01
 8.03891245e-01 8.18362256e-01 8.32833268e-01 8.47304280e-01
 8.6

In [87]:
np.interp(allrows_centers,[0,1],[internal_settings["geo"]["north"],internal_settings["geo"]["south"]])

array([48.0195912 , 48.01129569, 47.99552229, 47.97974889, 47.96397548,
       47.94820208, 47.93242868, 47.91665528, 47.90088187, 47.88510847,
       47.86933507, 47.85356166, 47.83778826, 47.82201486, 47.80624146,
       47.79046805, 47.77469465, 47.75892125, 47.74314785, 47.72737444,
       47.71160104, 47.69582764, 47.68005424, 47.66428083, 47.64850743,
       47.63273403, 47.61696062, 47.60118722, 47.58541382, 47.56964042,
       47.55386701, 47.53809361, 47.52232021, 47.50654681, 47.4907734 ,
       47.475     , 47.4592266 , 47.44345319, 47.42767979, 47.41190639,
       47.39613299, 47.38035958, 47.36458618, 47.34881278, 47.33303938,
       47.31726597, 47.30149257, 47.28571917, 47.26994576, 47.25417236,
       47.23839896, 47.22262556, 47.20685215, 47.19107875, 47.17530535,
       47.15953195, 47.14375854, 47.12798514, 47.11221174, 47.09643834,
       47.08066493, 47.06489153, 47.04911813, 47.03334472, 47.01757132,
       47.00179792, 46.98602452, 46.97025111, 46.95447771, 46.93

In [5]:
# Check all tif-files inside the intermediary folder

temporary_settings["existent_files"] = []

with os.scandir(internal_settings["files"]["intermediary_folder"]) as dirlist:
    temporary_settings["existent_files"] = [file.path for file in dirlist if re.search(r"(?i)\.tif", file.name)]

print ( temporary_settings["existent_files"] )



[]


In [6]:
# Before calling the api again, we should check if there’s a file existing that is exactly like the query 
for file in temporary_settings["existent_files"]:
    with rasterio.open(file) as src:
        average_error = (abs(src.bounds.left - internal_settings["geo"]["west"]) + abs(src.bounds.right - internal_settings["geo"]["east"]) + abs(src.bounds.top - internal_settings["geo"]["north"]) + abs(src.bounds.bottom - internal_settings["geo"]["south"])) / 4

        if average_error < internal_settings["geo"]["tolerance"]: temporary_settings["dem_already_existent"] = True

In [7]:
# Get the API key into the request data
api_call_data = json.loads(json.dumps(internal_settings["geo"]))
api_call_data.update({"API_Key": external_settings["opentopo_api_key"]})


In [8]:
if temporary_settings["dem_already_existent"]:
        print("Comparable DEM file already exists, skipping API call.")
else:
    response = requests.get("https://portal.opentopography.org/API/globaldem", params=api_call_data)
    with open(temporary_settings["dem_file_location"], "wb") as file:
        file.write(response.content)


# Aufteilen
Die DEM Daten in Kacheln aufteilen, die 100 km groß sind

In [ ]:
# How to make the GeoTIFF images usable by my algorithm:
# It’s important to note, that the original DEM data is in arc seconds
# To avoid skewed results it has to be compressed by cos(latitude) in width
# That way the metric distances (almost) resemble the distances pixel-wise

# After that the images will be scaled to 1/4 (as of now), because processing 
# becomes about 4^2 times faster.

# Before processing, the data will have to be split up into quadratic tiles
# which are supposed to be equal in their length.

# I guess we should combine the processes, namely selecting tiles of equal size
# and then reducing their size to a given size in pixels